In [ ]:
!pip install jieba 

In [ ]:
!pip install jieba 

In [ ]:
!pip install matplotlib

In [ ]:
!pip install WordCloud

In [ ]:
!pip install ImageColorGenerator

In [ ]:
import pandas as pd
import numpy as np
import jieba
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
from wordcloud import WordCloud, ImageColorGenerator # 词云包

In [ ]:
comm = pd.read_csv('D:/python_test_6_data/zhanlang_comment.csv', encoding='gbk') 
comm.head()

In [ ]:
# 数据的基本信息
comm.info()

In [ ]:
comm.shape

In [ ]:
# 去除 Unamed 列
comm.drop('Unnamed: 0', axis =1, inplace = True)

In [ ]:
# 统计空值数量 发现不存在空值
comm.isnull().sum()

In [ ]:
# 重新命名索引值 serices结构
comm.index = np.arange(len(comm))

In [ ]:
# 导入停用词表
stopwords = pd.read_csv('D:/python_test_6_data/stopword.txt', index_col = False, quoting = 3, sep = "\t", names = ['stopword'], encoding = 'utf-8')  
stopwords.head()

In [ ]:
# 分词 将一句话分成若干个词语
def cutDescription(x):
    seg_lst = jieba.lcut_for_search(x, HMM = True)
    return seg_lst

In [ ]:
# 去除停用词
def eliminateStopwords(x):
    words = pd.DataFrame({'segment' : x})
    keywords = words[-words['segment'].isin(stopwords['stopword'])]
    return list(keywords['segment'])

In [ ]:
# 利用pandas中的apply方法 调用预先定义的分词函数
comm['comment_seg'] = comm['comments'].apply(cutDescription)
comm.head()

In [ ]:
# 在分完词的列表中 继续去除停用词
comm['comment_seg_eli'] = comm['comment_seg'].apply(eliminateStopwords)
comm.head()

In [ ]:
# 经过分词 去除停用词后的所有词语放到一个列表中
all_words = []
for i in comm.index:
    all_words.extend(comm.loc[i,'comment_seg_eli'])
all_words

#### 分析结果的词云可视化展现

In [ ]:
pd.set_option('display.width', 1000)  # 设置字符显示宽度
pd.set_option('display.max_rows', None)  # 设置显示最大


In [ ]:
# 定义一个空的df 用来存放所有的词语
words_df = pd.DataFrame({'all_words' : all_words})

In [ ]:
# 分组聚合 统计每个词语出现的次数
# words_stat = words_df.groupby(by = ['all_words'])['all_words'].agg(['count'])
words_stat = words_df.groupby(by=['all_words'])['all_words'].count().to_frame('count').reset_index().sort_values(by=['count'], ascending=False)

In [ ]:
# 按照出现的次数进行排序 从大到小
words_stat = words_stat.reset_index().sort_values(by =['count'], ascending = False)

In [ ]:
# 去除所有空值
words_stat = words_stat[~words_stat.isnull().any(axis=1)]
words_stat.head(1000)

In [ ]:
# 去掉索引值为1 和 43 的行
words_stat.drop([1,43], inplace = True)
words_stat.head(1000)

In [ ]:
# 定义作图的参数
mpl.rc('figure', figsize = (10,5)) # 图的大小

In [ ]:
# 生成一个词云对象
wordcloud = WordCloud(font_path='D:/python_test_6_data/huakang.ttc', background_color='white', max_font_size=80)

# 将出现频率排名前1000的词语统计到词语频率表中
word_frequence = {x[1]: x[2] for x in words_stat.head(1000).values}

# 生成词云
wordcloud = wordcloud.fit_words(word_frequence)
plt.imshow(wordcloud)
plt.savefig('wordcloud.png')

- 使用read_csv读取数据时，提示"UnicodeDecodeError: 'utf-8' codec can't decode byte 0xce in position 13: invalid continuation byte"错误，参数中增加"encoding='gbk" 即可。
- 生成词云图时，出现"ValueError: anchor not supported for multiline text"报错，经过检查数据和查询资料，发现是由于索引为43的行，词值是"\n" ，删除这行记录后即可正常运行
- 生成的词云图，低频率的词反而尺寸大，检查代码后发现，是在排序时，错误的把索引号的列作为频率值的列统计到词语频率表中，修改后正常生成词云图。